In [1]:
#based on
#https://machinelearningmastery.com/time-series-forecasting-long-short-term-memory-network-python/
#https://datamarket.com/data/set/22r0/sales-of-shampoo-over-a-three-year-period#!ds=22r0&display=line

#Transform Time Series to Scale

In [2]:
DATA_DIR = "/home/ubuntu/fastai/data/"

In [3]:
# load and plot dataset
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime as pddatetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
from matplotlib import pyplot
import numpy


from datetime import datetime

Using TensorFlow backend.


In [4]:
# load dataset
def parser(x):
    return pddatetime.strptime('190'+x, '%Y-%m')


In [5]:
# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
	df = DataFrame(data)
	columns = [df.shift(i) for i in range(1, lag+1)]
	columns.append(df)
	df = concat(columns, axis=1)
	df.fillna(0, inplace=True)
	return df


In [6]:
# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return Series(diff)


In [7]:
# invert differenced value
def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]


In [8]:
# scale train and test data to [-1, 1]
def scale(train, test):
    #does this guarantee test_scaled will be within -1, 1 range?
    #since scale is set from train data not test.
	# fit scaler
	scaler = MinMaxScaler(feature_range=(-1, 1))
	scaler = scaler.fit(train)
	# transform train
	train = train.reshape(train.shape[0], train.shape[1])
	train_scaled = scaler.transform(train)
	# transform test
	test = test.reshape(test.shape[0], test.shape[1])
	test_scaled = scaler.transform(test)
	return scaler, train_scaled, test_scaled


In [9]:
# inverse scaling for a forecasted value
def invert_scale(scaler, X, value):
	new_row = [x for x in X] + [value]
	array = numpy.array(new_row)
	array = array.reshape(1, len(array))
	inverted = scaler.inverse_transform(array)
	return inverted[0, -1]


In [10]:
# fit an LSTM network to training data
def fit_lstm(train, batch_size, nb_epoch, neurons):
    #NB: The batch_size must be set to 1. This is because it must be a factor of the size of the training and test datasets.
    #look at this when fit_lstm is called.
	X, y = train[:, 0:-1], train[:, -1]
	X = X.reshape(X.shape[0], 1, X.shape[1])
	model = Sequential()
	model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
	model.add(Dense(1))
	model.compile(loss='mean_squared_error', optimizer='adam')
	for i in range(nb_epoch):
		model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
		model.reset_states()
	return model


In [11]:
# make a one-step forecast
def forecast_lstm(model, batch_size, X):
	X = X.reshape(1, 1, len(X))
	yhat = model.predict(X, batch_size=batch_size)
	return yhat[0,0]


In [12]:
series = read_csv(DATA_DIR+'shampoo_sales.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
# summarize first few rows
print(series.head())
print (series.shape)

Month
1901-01-01    266.0
1901-02-01    145.9
1901-03-01    183.1
1901-04-01    119.3
1901-05-01    180.3
Name: Sales, dtype: float64
(36,)


In [13]:
# transform data to be stationary
raw_values = series.values
diff_values = difference(raw_values, 1)
#
print ("raw_values:", type(raw_values), raw_values.shape)
print ("diff_values:", type(diff_values), diff_values.shape)

#we now have 1 less row since we calculated difference between current and immediately previous value.
#nbb: extra work would be required to create data over multiple time periods back.

raw_values: <class 'numpy.ndarray'> (36,)
diff_values: <class 'pandas.core.series.Series'> (35,)


In [14]:
# transform data to be supervised learning
supervised = timeseries_to_supervised(diff_values, 1)
supervised_values = supervised.values
print ("supervised_values:", type(supervised_values), supervised_values.shape)
#nb: we now have 2 columns since we are comparing sequence of data over a time period.

supervised_values: <class 'numpy.ndarray'> (35, 2)


In [15]:
# split data into train and test-sets
train = supervised_values[0:-12]
test  = supervised_values[-12:]


In [ ]:
print ("train:", train.shape)
print ("test:", test.shape)
# transform the scale of the data
scaler, train_scaled, test_scaled = scale(train, test)
#def scale : scale train and test data to [-1, 1]
print ("train_scaled:", train_scaled.shape, type(train_scaled))
print ("test_scaled:", test_scaled.shape, type(test_scaled))
print ("test_scaled:", numpy.amin(test_scaled), numpy.amax(test_scaled))
print ("train_scaled:", numpy.amin(train_scaled), numpy.amax(train_scaled))

train: (23, 2)
test: (12, 2)
train_scaled: (23, 2) <class 'numpy.ndarray'>
test_scaled: (12, 2) <class 'numpy.ndarray'>
test_scaled: -1.26760183437 1.32802805503
train_scaled: -1.0 1.0


In [ ]:
startTime= datetime.now()
print ("startTime:", startTime)

# repeat experiment
repeats = 30
error_scores = list()
for r in range(repeats):
	# fit the model
	lstm_model = fit_lstm(train_scaled, 1, 3000, 4)
	# forecast the entire training dataset to build up state for forecasting
	train_reshaped = train_scaled[:, 0].reshape(len(train_scaled), 1, 1)
	lstm_model.predict(train_reshaped, batch_size=1)
	# walk-forward validation on the test data
	predictions = list()
	for i in range(len(test_scaled)):
		# make one-step forecast
		X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
		yhat = forecast_lstm(lstm_model, 1, X)
		# invert scaling
		yhat = invert_scale(scaler, X, yhat)
		# invert differencing
		yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
		# store forecast
		predictions.append(yhat)
	# report performance
	rmse = sqrt(mean_squared_error(raw_values[-12:], predictions))
	print('%d) Test RMSE: %.3f' % (r+1, rmse))
	timeElapsed=datetime.now()-startTime
	print('Time elapsed (hh:mm:ss.ms) {}'.format(timeElapsed))
	error_scores.append(rmse)

    
timeElapsed=datetime.now()-startTime
print('Time elapsed (hh:mm:ss.ms) {}'.format(timeElapsed))

1) Test RMSE: 143.798
2) Test RMSE: 124.299
3) Test RMSE: 121.768
4) Test RMSE: 120.600
5) Test RMSE: 68.656
6) Test RMSE: 85.270
7) Test RMSE: 144.546
9) Test RMSE: 109.885
10) Test RMSE: 95.296


In [ ]:
pyplot.plot(predictions - raw_values[-12:])
pyplot.show()